# Online Surrogate Model Dashboard Demo

Note: Complete Server.ipynb notebook before moving forward with dashboard.

## Project description 

In this demo, we will launch a PVAccess server and use its process variables as the inputs and outputs of an online surrogate model. We will then create an application using sliders to control the input process variables, and a number of data views to capture the ouput variables.
<br>
*This would be a nice place to add some specific info about the actual modeling, descriptions of inputs/outputs etc.*
<br>


## Structure of the application folder




## Note on running- requires active server
Possible to run from another jupyter notebook kernel but could get weird with ports etc.
Recommend running command line server. 




## Description of application
For this demo, we will create a dashboard view of the MySurrogateModel class provided by Lipi Gupta.
<br>
The application consists of: 
<br>
Input control sliders
<br>
Image view
<br>
Value table
<br>
Striptool
<br>

Each of these items will be referred to as "widgets". 


In [1]:
# MAKE SURE THAT THE REPOSITORY ROOT IS IN THE PYTHONPATH
import sys
import os

module_path = os.path.abspath(os.path.join(os.pardir, os.pardir))
if module_path not in sys.path:
    sys.path.append(module_path)

In [2]:
from bokeh.io import output_notebook, show
from bokeh.models.widgets import Select
from bokeh import palettes
from bokeh.layouts import column, row, Spacer

output_notebook()

Loading BokehJS ...

In [3]:
from online_model.app.controllers import Controller
from online_model.app.widgets.sliders import build_sliders
from online_model.app.widgets.plots import ImagePlot, Striptool
from online_model.app.widgets.tables import ValueTable
from online_model import PREFIX, SIM_PVDB, CMD_PVDB, EXCLUDE_SLIDERS

# need surrogate model image processing methods
from online_model.model.MySurrogateModel import MySurrogateModel

<built-in function chdir>


OSError: Unable to open file (unable to open file: name = 'online_model/files/CNN_051620_SurrogateModel.h5', errno = 2, error message = 'No such file or directory', flags = 0, o_flags = 0)

The process variable databases are constructed when the __init___ method is called from the online_model folder. This populates the inputs and ouptuts using the data provided in the h5 file.

The widget tools use the information provided in the 

In [7]:
print(CMD_PVDB) #inputs

{'laser_radius': {'type': 'float', 'prec': 8, 'value': 0.34798698, 'units': 'mm', 'range': [0.1, 0.5]}, 'maxb(2)': {'type': 'float', 'prec': 8, 'value': 0.0402751972, 'units': 'T', 'range': [0.0, 0.1]}, 'phi(1)': {'type': 'float', 'prec': 8, 'value': -7.99101687, 'units': 'Degrees', 'range': [-10.0, 10.0]}, 'total_charge:value': {'type': 'float', 'prec': 8, 'value': 141.576322, 'units': 'pC', 'range': [0.0, 300.0]}, 'in_xmin': {'type': 'float', 'prec': 8, 'value': -0.000353964583, 'units': 'm', 'range': [-0.0004216, 0.0003977]}, 'in_xmax': {'type': 'float', 'prec': 8, 'value': 0.000344330666, 'units': 'm', 'range': [-0.0004216, 0.0003977]}, 'in_ymin': {'type': 'float', 'prec': 8, 'value': -0.000347874295, 'units': 'm', 'range': [-0.1117627, 0.1120053]}, 'in_ymax': {'type': 'float', 'prec': 8, 'value': 0.000345778376, 'units': 'm', 'range': [-0.1117627, 0.1120053]}}


In [8]:
print(SIM_PVDB) #outputs

{'end_core_emit_95percent_x': {'type': 'float', 'prec': 8, 'units': 'mm-mrad'}, 'end_core_emit_95percent_y': {'type': 'float', 'prec': 8, 'units': 'mm-mrad'}, 'end_core_emit_95percent_z': {'type': 'float', 'prec': 8, 'units': 'mm-mrad'}, 'end_mean_kinetic_energy': {'type': 'float', 'prec': 8, 'units': 'eV'}, 'end_mean_x': {'type': 'float', 'prec': 8, 'units': 'mm'}, 'end_mean_y': {'type': 'float', 'prec': 8, 'units': 'mm'}, 'end_n_particle_loss': {'type': 'float', 'prec': 8, 'units': 'Number'}, 'end_norm_emit_x': {'type': 'float', 'prec': 8, 'units': 'mm-mrad'}, 'end_norm_emit_y': {'type': 'float', 'prec': 8, 'units': 'mm-mrad'}, 'end_norm_emit_z': {'type': 'float', 'prec': 8, 'units': 'mm-mrad'}, 'end_sigma_x': {'type': 'float', 'prec': 8, 'units': 'mm'}, 'end_sigma_xp': {'type': 'float', 'prec': 8, 'units': 'mrad'}, 'end_sigma_y': {'type': 'float', 'prec': 8, 'units': 'mm'}, 'end_sigma_yp': {'type': 'float', 'prec': 8, 'units': 'mrad'}, 'end_sigma_z': {'type': 'float', 'prec': 8, 'un

# Set up the controller

In [9]:
PROTOCOL = "pva"

# create controller
controller = Controller(PROTOCOL)

# SLIDERS

In [10]:
# build sliders for the command process variable database
# TEMPORARILY EXCLUDE THE EXTENTS
sliders_to_render = {}
for var, value in CMD_PVDB.items():
    if "in_" not in EXCLUDE_SLIDERS:
        sliders_to_render[var] = value

sliders = build_sliders(sliders_to_render, controller)

# IMAGE
Two associated items:
<br>
Selection

## Callbacks
Variable selection (are we looking at x vs. y, x vs. z, etc.)
<br>
Periodic data update


In [11]:
# Create custom palette with low values set to white
pal = list(palettes.viridis(244))  # 256 - 12 (set lowest 5% to white)
pal = ["#FFFFFF"] * 12 + pal
pal = tuple(pal)

# create plot
# need to pass MySurrogateModel class into the plot because need access to the prepare_image_from_pv method
image_plot = ImagePlot(SIM_PVDB, controller, MySurrogateModel)
image_plot.build_plot(pal)

# track current_pv globally
current_image_pv = image_plot.current_pv

# set up image toggle
image_select = Select(
    title="Image PV",
    value=current_image_pv,
    options=list(image_plot.pv_monitors.keys()),
)

# callback to update the image variables
def on_image_selection(attrname, old, new):
    """
    Callback function for dropdown selection that updates the global current variable.
    """
    global current_image_pv
    current_image_pv = new

#assign the callback to the image selection
image_select.on_change("value", on_image_selection)

# Set up image update callback
def image_update_callback():
    """
    Calls plot controller update with the current global process variable
    """
    global current_image_pv
    image_plot.update(current_image_pv)

# STRIPTOOL

In [12]:
# Set up the striptool
striptool = Striptool(SIM_PVDB, controller)
striptool.build_plot()

# set up global pv
current_striptool_pv = striptool.current_pv

# create a selection tool so we can switch between output variables
striptool_select = Select(
    title="PV to Plot:",
    value=current_striptool_pv,
    options=list(striptool.pv_monitors.keys()),
)

# create a selection callback
def striptool_select_callback(attr, old, new):
    global current_striptool_pv
    current_striptool_pv = new
    
# striptool data update callback
def striptool_update_callback():
    """
    Calls plot controller update with the current global process variable
    and updates the value table.
    """
    global current_striptool_pv
    striptool.update(current_striptool_pv)
    value_table.update()

# assign the selection callback to the striptool
striptool_select.on_change("value", striptool_select_callback)

# Value table

In [13]:
# add table
value_table = ValueTable(SIM_PVDB, controller)

# Set up callback to update the table
def table_update_callback():
    """
    Updates the value table.
    """
    value_table.update()

# Set up our application layout

# NOTE:
Depending on the order in which you opened your server and dashboard notebooks, you may run into a bokeh error, :



If you attempt to run the following code without stopping the kernel, you will run into the following error:
``RuntimeError: Models must be owned by only a single document``

In [17]:
def render_app(doc):
    """
    Function for rendering the application within the embedded bokeh server.
    """
    doc.title = "Online Surrogate Model Image Viewer"
    doc.add_root( column(
        row(column(sliders, width=350), Spacer(width=50),  column(value_table.table, height=300)),  # add sliders
        row(column(image_select, image_plot.p), column(striptool_select, striptool.p))
    ))# add image controls)
    doc.add_periodic_callback(image_update_callback, 250)
    doc.add_periodic_callback(striptool_update_callback, 250)
    doc.add_periodic_callback(table_update_callback, 250)

    
show(render_app)

No process variable found for smvm:x:y
No process variable found for smvm:end_core_emit_95percent_x
No process variable found for smvm:end_core_emit_95percent_y
No process variable found for smvm:end_core_emit_95percent_z
No process variable found for smvm:end_mean_kinetic_energy
No process variable found for smvm:end_mean_x
No process variable found for smvm:end_mean_y
No process variable found for smvm:end_n_particle_loss
No process variable found for smvm:end_norm_emit_x
No process variable found for smvm:end_norm_emit_y
No process variable found for smvm:end_norm_emit_z
No process variable found for smvm:end_sigma_x
No process variable found for smvm:end_sigma_xp
No process variable found for smvm:end_sigma_y
No process variable found for smvm:end_sigma_yp
No process variable found for smvm:end_sigma_z
No process variable found for smvm:end_total_charge
No process variable found for smvm:out_xmin
No process variable found for smvm:out_xmax
No process variable found for smvm:out_ymi

No process variable found for smvm:end_total_charge
No process variable found for smvm:out_xmin
No process variable found for smvm:out_xmax
No process variable found for smvm:out_ymin
No process variable found for smvm:out_ymax
No process variable found for smvm:x:y
No process variable found for smvm:end_core_emit_95percent_x
No process variable found for smvm:end_core_emit_95percent_y
No process variable found for smvm:end_core_emit_95percent_z
No process variable found for smvm:end_mean_kinetic_energy
No process variable found for smvm:end_mean_x
No process variable found for smvm:end_mean_y
No process variable found for smvm:end_n_particle_loss
No process variable found for smvm:end_norm_emit_x
No process variable found for smvm:end_norm_emit_y
No process variable found for smvm:end_norm_emit_z
No process variable found for smvm:end_sigma_x
No process variable found for smvm:end_sigma_xp
No process variable found for smvm:end_sigma_y
No process variable found for smvm:end_sigma_yp
N

In [15]:
os.environ["BOKEH_ALLOW_WS_ORIGIN"] = "localhost:8889"